In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from datenspende.utils import query_ch_df, query_pg_df
import datetime
from datetime import date
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import re
import os
import glob
from functools import reduce

In [2]:
lc_u = np.load('uid_per_shb_fatigue.npy')

In [3]:
lc_u2=np.load('uid_per_shb_fatigue(1).npy')

In [4]:
path = os.getcwd()+'/user_df/'
csv_files = glob.glob(os.path.join(path, "*.csv"))

df_lcu = []
demo_info = []
uid = []
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f, index_col = 0)
    for var in ['sleep', 'hr', 'steps']:
        if var not in df.columns:
            df[var] = np.nan
    df_lcu.append(df)
    demo_info.append(f.split("/")[-1].split('.')[0])
    uid.append(int(f.split("/")[-1].split('.')[1][1:])) 
      
sex = [re.split('(\d+)',demo_info[i])[0] for i in range(len(demo_info))]
age = [int(re.split('(\d+)',demo_info[i])[1]) for i in range(len(demo_info))] 

df_lcu.pop(31)
uid.pop(31)
sex.pop(31)
age.pop(31)
df_lcu.pop(67)
uid.pop(67)
sex.pop(67)
age.pop(67)

60

In [5]:
    # group by day and hourly - sum up steps and take mean of heart rate
    # per day 24 time bins with steps summed up and mean of heart rate
    df_lcu_agg = []
    #for du in range(len(df_lcu[0])): 
    d = df_lcu[22].copy()
    d['steps'] = d['steps'].fillna(0)
    d['steps'] = d['steps'] + 1
    
    d_agg = pd.DataFrame(columns = ['hr', 'd to max hr [%]', 'date', 'time','weekend', 'week_totest', 'phase' ])
    
    for day in d['date'].unique():
        d_day = d[d['date'] ==day]
        d_day_agg_m = d_day[['time', 'hr', 'd to max hr [%]']].groupby('time').mean().groupby(np.arange(96)//4).mean()
        
        hr = d_day_agg_m.loc[:,'hr']
         
        
        
        d_day_agg_m['date'] = day
        d_day_agg_m['time'] = d_day_agg_m.index
        d_day_agg_m['weekend'] = d_day['weekend'].iloc[0]
        d_day_agg_m['week_totest'] = d_day['week_totest'].iloc[0]
        d_day_agg_m['phase'] = d_day['phase'].iloc[0]
         
         
        
        
        d_agg = d_agg.append(d_day_agg_m)
    d_agg = d_agg.reset_index()
    df_lcu_agg.append(d_agg)

In [9]:
mean_h = d_agg[d_agg['week_totest'] < -1].groupby('index').mean()

In [16]:
import sys
!{sys.executable} -m pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 41.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [17]:
from scipy.stats import mannwhitneyu

In [26]:
from scipy import stats

In [29]:
for d in d_agg['date'].unique():
    dd = d_agg[d_agg['date']==d]
    val = dd.groupby('index').mean() - mean_h 
    print(stats.ttest_ind(val['hr'], mean_h['hr'] )) 

Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indRe

In [19]:
mannwhitneyu(val['hr'], mean_h['hr'], alternative="less", method="exact")

MannwhitneyuResult(statistic=nan, pvalue=nan)

In [22]:
mean_h 

,hr,d to max hr [%]
index,,
0,73.829717,44.368820
1,68.601770,41.227025
2,66.300117,39.843821
3,64.125615,38.537028
4,65.736104,39.504870
5,67.566880,40.605096
6,73.073509,43.914369
7,77.591632,46.629587
8,80.857034,48.591968
